# 重复样本训练方法

In [18]:
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score,precision_score, recall_score, f1_score
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree

In [2]:
dataset = pd.read_csv(r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter_havelabel.csv")
dataset.dropna(inplace=True)

In [3]:
dataset = dataset[dataset['label'] == 'fish_net']
dataset= dataset.drop(columns=['label'])

In [4]:
dataset

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,SRA,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL
0,13d,1021.6,0.186124,5.645218,940.00836,81.591610,0.000000,4.197873,13.658210,0.010529,...,13140.667,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7
1,13d,701.7,1.216353,2.768598,694.73010,6.969910,0.000000,-1.750685,9.647439,-0.005612,...,13003.167,16.391666,1.587500,1.583333,191.06468,472945.0,2907200.0,6.843954,4,3
2,13d,661.0,0.785157,5.674496,663.14667,0.000000,2.146667,1.533146,0.001766,0.000064,...,13015.000,16.375000,1.588333,1.583333,191.17732,473845.0,2907200.0,6.713590,4,39
3,13d,869.8,0.820064,3.988137,857.71760,12.082397,0.000000,2.137851,13.153590,0.005155,...,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861600.0,5.598476,6,3
4,13d,1031.5,0.443475,4.917029,903.36255,128.137450,0.000000,0.071758,14.658850,0.000746,...,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861900.0,5.441568,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,1e,1646.3,1.134341,1.592065,1615.18730,31.112793,0.000000,1.626131,14.525656,0.003527,...,12672.333,12.325000,1.260000,2.100000,192.85692,434245.0,2918000.0,5.110536,3,7
4388,1e,1600.5,0.950942,1.269871,1560.65940,39.840576,0.000000,-0.021284,9.449365,0.000551,...,12697.000,12.758333,1.300000,2.000000,191.90393,434545.0,2918000.0,5.209802,3,7
4389,1e,1656.4,0.719550,0.394788,1613.64420,42.755860,0.000000,-2.737058,22.095870,-0.005495,...,12677.667,12.483334,1.274167,2.066667,192.62935,433345.0,2918300.0,5.140053,3,7
4390,1e,1630.9,0.629897,0.141897,1615.21340,15.686645,0.000000,-0.026897,13.455787,-0.000136,...,12672.333,12.325000,1.260000,2.100000,192.85692,433645.0,2918300.0,5.111474,3,7


In [5]:
# 标签化
dataset["TZ_label"] = dataset.TZ.astype("category").cat.codes
result = dataset.groupby('TZ_label')["TZ"].apply(lambda x: list(x.unique())).to_dict()
print(result)

{0: ['101'], 1: ['102'], 2: ['103'], 3: ['104'], 4: ['105'], 5: ['106'], 6: ['107'], 7: ['108'], 8: ['112'], 9: ['113'], 10: ['114'], 11: ['115'], 12: ['116'], 13: ['117'], 14: ['118'], 15: ['119'], 16: ['11e'], 17: ['120'], 18: ['125'], 19: ['126'], 20: ['129'], 21: ['12b'], 22: ['12c'], 23: ['12d'], 24: ['12e'], 25: ['12f'], 26: ['130'], 27: ['131'], 28: ['132'], 29: ['135'], 30: ['136'], 31: ['137'], 32: ['138'], 33: ['139'], 34: ['13d'], 35: ['13e'], 36: ['13f'], 37: ['141'], 38: ['143'], 39: ['14d'], 40: ['14e'], 41: ['14f'], 42: ['15'], 43: ['15d'], 44: ['15e'], 45: ['15f'], 46: ['18d'], 47: ['1e'], 48: ['1f'], 49: ['21'], 50: ['22'], 51: ['23'], 52: ['24'], 53: ['26'], 54: ['29'], 55: ['2b'], 56: ['2f'], 57: ['30'], 58: ['31'], 59: ['32'], 60: ['33'], 61: ['34'], 62: ['4d'], 63: ['4e'], 64: ['4f'], 65: ['61d'], 66: ['61e'], 67: ['61f'], 68: ['63'], 69: ['63b'], 70: ['63d'], 71: ['63e'], 72: ['63f'], 73: ['64d'], 74: ['64f'], 75: ['65'], 76: ['65d'], 77: ['65e'], 78: ['67'], 79: 

In [6]:
dataset['DZ']=dataset['DZ'].astype('category')
dataset['DL']=dataset['DL'].astype('category')
dataset['TZ_label']=dataset['TZ_label'].astype('category')

In [7]:
features = dataset.columns[1:-1]
target = "TZ_label"

In [8]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

In [21]:
# 将结果转换为 pandas 数据框
class_counts_df = pd.DataFrame(class_counts)
class_counts_df.columns = ['sample_count']

# 添加类别名称和样本百分比
class_counts_df['class_name'] = class_counts_df.index
class_counts_df['class_percentage'] = class_counts_df['sample_count'] / class_counts_df['sample_count'].sum()

# 输出到 CSV 文件
class_counts_df.to_csv(r"C:\Users\Runker\Desktop\result.csv", index=False)

In [20]:
class_counts

TZ_label
39    693
25    662
34    514
24    407
41    271
     ... 
32      1
37      1
83      1
54      1
49      1
Name: count, Length: 99, dtype: int64

In [10]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

# 找出样本数少于2的类别
minority_classes = class_counts[class_counts <= 15].index.tolist()

# 初始化一个空的DataFrame来存储增加的样本
additional_samples = pd.DataFrame(columns=dataset.columns)

# 对每个少数类别重复样本，直到至少有15个样本
for class_label in minority_classes:
    # 获取当前类别的所有样本
    class_samples = dataset[dataset['TZ_label'] == class_label]
    print(class_label)
    # 检查是否存在样本
    if not class_samples.empty:
        # 计算需要增加多少样本
        samples_to_add = 15 - len(class_samples)
        print(class_label)
        # 检查是否需要增加样本
        if samples_to_add > 0:
            # 重复样本
            resampled = resample(class_samples,
                                 replace=True,  # 允许选择相同的样本多次
                                 n_samples=samples_to_add,  # 需要增加的样本数量
                                 random_state=123)  # 随机状态以确保可复现性
            # 将增加的样本添加到additional_samples
            additional_samples = pd.concat([additional_samples, resampled], axis=0)

# 将增加的样本添加回原始数据集
data_augmented = pd.concat([dataset, additional_samples], axis=0).reset_index(drop=True)

# 现在，data_augmented 包含了原始数据和增加的样本，可以用于进一步的分析或模型训练


17
17
12
12
47
47
85
85
14
14
36
36
60
60
0
0
20
20
68
68
95
95
91
91
72
72
56
56
27
27
90
90
62
62
69
69
58
58
84
84
50
50
1
1
61
61
11
11
29
29
55
55
4
4
59
59
48
48
19
19
6
6
80
80
94
94
74
74
30
30
86
86
82
82
51
51
89
89
78
78
97
97
87
87
98
98
42
42
13
13
33
33
65
65
73
73
28
28
3
3
8
8
10
10
93
93
15
15
16
16
31
31
38
38
52
52
79
79
88
88
18
18
32
32
37
37
83
83
54
54
49
49


C:\Users\Runker\AppData\Local\Temp\ipykernel_8236\3857064541.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
C:\Users\Runker\AppData\Local\Temp\ipykernel_8236\3857064541.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
C:\Users\Runker\AppData\Local\Temp\ipykernel_8236\3857064541.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. 

In [11]:
data_augmented

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,TZ_label
0,13d,1021.6,0.186124,5.645218,940.00836,81.591610,0.000000,4.197873,13.658210,0.010529,...,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7,34
1,13d,701.7,1.216353,2.768598,694.73010,6.969910,0.000000,-1.750685,9.647439,-0.005612,...,16.391666,1.587500,1.583333,191.06468,472945.0,2907200.0,6.843954,4,3,34
2,13d,661.0,0.785157,5.674496,663.14667,0.000000,2.146667,1.533146,0.001766,0.000064,...,16.375000,1.588333,1.583333,191.17732,473845.0,2907200.0,6.713590,4,39,34
3,13d,869.8,0.820064,3.988137,857.71760,12.082397,0.000000,2.137851,13.153590,0.005155,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861600.0,5.598476,6,3,34
4,13d,1031.5,0.443475,4.917029,903.36255,128.137450,0.000000,0.071758,14.658850,0.000746,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861900.0,5.441568,3,7,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,21,989.4,1.202222,2.988946,962.59980,26.800232,0.000000,3.272304,11.206271,0.008857,...,15.474999,1.526667,1.675000,193.85529,445645.0,2870300.0,5.428254,9,9,49
4980,21,989.4,1.202222,2.988946,962.59980,26.800232,0.000000,3.272304,11.206271,0.008857,...,15.474999,1.526667,1.675000,193.85529,445645.0,2870300.0,5.428254,9,9,49
4981,21,989.4,1.202222,2.988946,962.59980,26.800232,0.000000,3.272304,11.206271,0.008857,...,15.474999,1.526667,1.675000,193.85529,445645.0,2870300.0,5.428254,9,9,49
4982,21,989.4,1.202222,2.988946,962.59980,26.800232,0.000000,3.272304,11.206271,0.008857,...,15.474999,1.526667,1.675000,193.85529,445645.0,2870300.0,5.428254,9,9,49


In [12]:
# 划分数据集
X = data_augmented[features]
y = data_augmented[target]
# 分割训练集和测试集
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# 通过网格搜索找到最优超参数
params = {'criterion':['gini','entropy'], 'max_depth':list(range(2,100,10))} 
grid_tree = GridSearchCV(DecisionTreeClassifier(), params, cv=StratifiedKFold(n_splits=5))
grid_tree.fit(Xtrain, ytrain)
print(grid_tree.best_params_)

{'criterion': 'entropy', 'max_depth': 92}


In [14]:

random_state = 42
tree_classifier = DecisionTreeClassifier(random_state=42,**grid_tree.best_params_)
# tree_classifier = DecisionTreeClassifier(random_state=42,criterion='gini', max_depth=5)
tree_classifier.fit(Xtrain, ytrain)

DecisionTreeClassifier(criterion='entropy', max_depth=92, random_state=42)

In [15]:
predictor = tree_classifier
# 查看准确率
y_test_pred= predictor.predict(Xtest)
y_train_pred = predictor.predict(Xtrain)
accuracy_test = accuracy_score(ytest,y_test_pred)
accuracy_train = accuracy_score(ytrain,y_train_pred)
print(accuracy_train,accuracy_test)

1.0 0.7502507522567703


In [16]:
# 决策树保存路径
model_path = r"F:\cache_data\model_path\dy\soil_type\cart"
model_name = "cart_tree.pkl"

In [17]:
# 保存模型
with open(os.path.join(model_path,model_name), 'wb') as f:
    pickle.dump(tree_classifier, f)

In [19]:
# 注意：由于是多分类问题，我们需要指定 average 参数。这里使用 "macro" 作为示例
# "macro" 未加权平均，"weighted" 会根据每个类别的样本数加权平均
precision = precision_score(ytest, y_test_pred, average='macro')
recall = recall_score(ytest, y_test_pred, average='macro')
f1 = f1_score(ytest, y_test_pred, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.7207772066608643
Recall: 0.7659997935238424
F1 Score: 0.7202951089342235
